In [13]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

file_path = 'processed_Исторические_данные_по_офертам_поставщиков_на_лот.csv'
data = pd.read_csv(file_path)

manual_lot_count = 4996
manual_avg_price = 910874

# Step 1: Preprocess the data
label_encoder = LabelEncoder()
data_preprocessed = pd.DataFrame()
# If there are categorical variables, you might want to one-hot encode them
# data_preprocessed = pd.get_dummies(data, columns=['Материал', 'Класс', 'Кредитор'], drop_first=True)
data_preprocessed['Материал'] = label_encoder.fit_transform(data['Материал'])
data_preprocessed['Класс'] = label_encoder.fit_transform(data['Класс'])
data_preprocessed['Кредитор'] = label_encoder.fit_transform(data['Кредитор'])
data_preprocessed['price'] = data['Сумма во ВВ'].copy()
print(data_preprocessed)

# Standardize the data to have a mean of 0 and variance of 1
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_preprocessed)

# Step 2: Apply K-Means clustering
k = 100  # Choose the number of clusters (lots)
kmeans = KMeans(n_clusters=k, random_state=42)
data['cluster'] = kmeans.fit_predict(data_scaled)

# data['Материал_Класс'] = data['Материал'].astype(str) + '-' + data['Класс'].astype(str)
data['Материал_Класс'] = data['Класс'].astype(str)

# Step 2: Add a column for the number of unique "Материал_Класс" pairs in each cluster
data['pairs_per_clust'] = data.groupby('cluster')['Материал_Класс'].transform('nunique')

# Step 3: Add a column to count the number of unique "Материал_Класс" pairs covered by each creditor in each cluster
data['pairs_by_cred'] = data.groupby(['Кредитор', 'cluster'])['Материал_Класс'].transform('nunique')

# Step 4: Calculate the percentage of pairs each creditor covers in each cluster
data['perc_cov'] = (data['pairs_by_cred'] / data['pairs_per_clust']) * 100

def categorize_coverage(percentage):
    if 80 > percentage >= 50:
        return '≥ 50%'
    elif 100 > percentage >= 80:
        return '≥ 80%'
    elif percentage == 100:
        return '100%'

data['cov_cat'] = data['perc_cov'].apply(categorize_coverage)

# Group by cluster and calculate the number of creditors in each category
creditor_counts_by_category = data.groupby(['cluster', 'cov_cat'])['Кредитор'].nunique().reset_index()
cred_count = data.groupby(['cluster', 'cov_cat'])['Кредитор'].transform('nunique')
data['num_creditors'] = cred_count
creditor_counts_by_category.columns = ['cluster', 'cov_cat', 'num_creditors']

# Calculate the average number of creditors per cluster in each coverage category
avg_creditors_per_category = creditor_counts_by_category.groupby('cov_cat')['num_creditors'].mean().reset_index()

print(avg_creditors_per_category)

# Extract values and store them in separate variables
average_creditors_le_50 = avg_creditors_per_category.loc[
    (avg_creditors_per_category['cov_cat'] == '≥ 50%') | 
    (avg_creditors_per_category['cov_cat'] == '≥ 80%') | 
    (avg_creditors_per_category['cov_cat'] == '100%'), 
    'num_creditors'
    ].values[0]
average_creditors_50_to_80 = avg_creditors_per_category.loc[
    (avg_creditors_per_category['cov_cat'] == '≥ 80%') |
    (avg_creditors_per_category['cov_cat'] == '100%'),
    'num_creditors'
    ].values[0]
average_creditors_ge_80 = avg_creditors_per_category.loc[avg_creditors_per_category['cov_cat'] == '100%', 'num_creditors'].values[0]
average_creditors_per_lot = data.groupby('cluster')['Кредитор'].nunique().mean()

# Step 3: Evaluate the result
# Number of lots
num_lots = data['cluster'].nunique()
print(f'Number of lots: {num_lots}')

# Calculate the average price per lot
average_price_per_lot = data.groupby('cluster')['Сумма во ВВ'].sum().mean()
print("Average price per lot:")
print(average_price_per_lot)

MQ = 0.5 * ((1 - num_lots / manual_lot_count) + (1 - manual_avg_price / average_price_per_lot))

MS = (2 * average_creditors_le_50 + 3 * average_creditors_50_to_80 + 4 * average_creditors_ge_80) / average_creditors_per_lot

print(f"MQ metric is: {MQ}")
print(f"MS metric is: {MS}")


        Материал  Класс  Кредитор      price
0              0     36       408     137.00
1              0     36       586     567.00
2              0     36       592     619.76
3              0     36        24     531.00
4              0     36       586     640.30
...          ...    ...       ...        ...
113603     10690      1       690   12234.00
113604     10690      1       756   11780.00
113605     10691     28       854   32000.00
113606     10692     12       854     249.17
113607     10693     36         0  186374.00

[113608 rows x 4 columns]
        Unnamed: 0        Лот ID      Кредитор  Сумма во ВВ      Материал  \
0                0  1.000005e+11  7.542200e+04       137.00       1118280   
1                1  1.000005e+11  9.429300e+04       567.00       1118280   
2                4  1.000005e+11  9.450100e+04       619.76       1118280   
3                5  1.000006e+11  2.328800e+04       531.00       1118280   
4                8  1.000006e+11  9.429300e+04  

In [6]:
data[data['cluster'] == 591]

,Unnamed: 0,Лот ID,Кредитор,Сумма во ВВ,Материал,Краткий текст,Влт,Курс,Краткий текст материала,Класс,Название,cluster,Материал_Класс,pairs_per_clust,pairs_by_cred,perc_cov
1,1,1.000005e+11,94293.0,567.00,1118280,Площадка самоклеящаяся Hyperline HW-3A,RUB,1.0,Площадка самоклеящаяся Hyperline HW-3A,G2170,ПРОДУКЦИЯ КАБЕЛЬНАЯ НЕ НОРМАЛИЗОВАННАЯ,591,1118280-G2170,7,1,14.285714
2,4,1.000005e+11,94501.0,619.76,1118280,Площадка самоклеящаяся Hyperline HW-3A,RUB,1.0,Площадка самоклеящаяся Hyperline HW-3A,G2170,ПРОДУКЦИЯ КАБЕЛЬНАЯ НЕ НОРМАЛИЗОВАННАЯ,591,1118280-G2170,7,1,14.285714
4,8,1.000006e+11,94293.0,640.30,1118280,Площадка самоклеящаяся Hyperline HW-3A,RUB,1.0,Площадка самоклеящаяся Hyperline HW-3A,G2170,ПРОДУКЦИЯ КАБЕЛЬНАЯ НЕ НОРМАЛИЗОВАННАЯ,591,1118280-G2170,7,1,14.285714
11,22,1.000007e+11,87239.0,0.00,1121640,Эмаль ПФ-115 желтая с.1,NaN,1.0,Кабель КВВГ 7х1 б/у,G2160,"Кабель б/у, провода б/у",591,1121640-G2160,7,1,14.285714
15,34,1.000007e+11,87239.0,141.00,1121640,Эмаль ПФ-115 желтая с.1,RUB,1.0,Кабель КВВГ 7х1 б/у,G2160,"Кабель б/у, провода б/у",591,1121640-G2160,7,1,14.285714
16,37,1.000007e+11,87239.0,130.00,1121640,Эмаль ПФ-115 желтая с.1,RUB,1.0,Кабель КВВГ 7х1 б/у,G2160,"Кабель б/у, провода б/у",591,1121640-G2160,7,1,14.285714
836,2262,1.000005e+11,91364.0,1950.00,1167016,Очиститель Scotchcast 3M/4413-S,RUB,1.0,Очиститель Scotchcast 3M/4413-S,G2170,ПРОДУКЦИЯ КАБЕЛЬНАЯ НЕ НОРМАЛИЗОВАННАЯ,591,1167016-G2170,7,1,14.285714
6148,16909,1.000007e+11,93259.0,36900.00,770000082380,Набор инструментов 158пр Hans/TK-158V,RUB,1.0,Кабель MIC5.K микрофонный RapcoHorizon,G2170,ПРОДУКЦИЯ КАБЕЛЬНАЯ НЕ НОРМАЛИЗОВАННАЯ,591,770000082380-G2170,7,1,14.285714
6542,17914,1.000007e+11,92855.0,66.95,770000096010,Карман самокл. Комус/166549,RUB,1.0,Нагреватель ILL312CF Quintex,G2110,Кабели греющие,591,770000096010-G2110,7,1,14.285714
6543,17915,1.000007e+11,92855.0,65.57,770000096010,Карман самокл. Комус/166549,RUB,1.0,Нагреватель ILL312CF Quintex,G2110,Кабели греющие,591,770000096010-G2110,7,1,14.285714


In [21]:
import tqdm
from tqdm import trange

file_path = 'processed_Исторические_данные_по_офертам_поставщиков_на_лот.csv'
data = pd.read_csv(file_path)

manual_lot_count = 4996
manual_avg_price = 910874

# Step 1: Preprocess the data
label_encoder = LabelEncoder()
data_preprocessed = pd.DataFrame()
# If there are categorical variables, you might want to one-hot encode them
# data_preprocessed = pd.get_dummies(data, columns=['Материал', 'Класс', 'Кредитор'], drop_first=True)
data_preprocessed['Материал'] = label_encoder.fit_transform(data['Материал'])
data_preprocessed['Класс'] = label_encoder.fit_transform(data['Класс'])
data_preprocessed['Кредитор'] = label_encoder.fit_transform(data['Кредитор'])
data_preprocessed['price'] = data['Сумма во ВВ'].copy()

# Standardize the data to have a mean of 0 and variance of 1
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_preprocessed)

MQ_max = 0.5
index = 0
for k in trange(100, manual_lot_count, 100):

    kmeans = KMeans(n_clusters=k, random_state=42)
    data['cluster'] = kmeans.fit_predict(data_scaled)

    # data['Материал_Класс'] = data['Материал'].astype(str) + '-' + data['Класс'].astype(str)
    data['Материал_Класс'] = data['Класс'].astype(str)

    # Step 2: Add a column for the number of unique "Материал_Класс" pairs in each cluster
    data['pairs_per_clust'] = data.groupby('cluster')['Материал_Класс'].transform('nunique')

    # Step 3: Add a column to count the number of unique "Материал_Класс" pairs covered by each creditor in each cluster
    data['pairs_by_cred'] = data.groupby(['Кредитор', 'cluster'])['Материал_Класс'].transform('nunique')

    # Step 4: Calculate the percentage of pairs each creditor covers in each cluster
    data['perc_cov'] = (data['pairs_by_cred'] / data['pairs_per_clust']) * 100

    def categorize_coverage(percentage):
        if 80 > percentage >= 50:
            return '≥ 50%'
        elif 100 > percentage >= 80:
            return '≥ 80%'
        elif percentage == 100:
            return '100%'

    data['cov_cat'] = data['perc_cov'].apply(categorize_coverage)

    # Group by cluster and calculate the number of creditors in each category
    creditor_counts_by_category = data.groupby(['cluster', 'cov_cat'])['Кредитор'].nunique().reset_index()
    cred_count = data.groupby(['cluster', 'cov_cat'])['Кредитор'].transform('nunique')
    data['num_creditors'] = cred_count
    creditor_counts_by_category.columns = ['cluster', 'cov_cat', 'num_creditors']
    avg_creditors_per_category = creditor_counts_by_category.groupby('cov_cat')['num_creditors'].mean().reset_index()

    average_creditors_le_50 = avg_creditors_per_category.loc[
        (avg_creditors_per_category['cov_cat'] == '≥ 50%') | 
        (avg_creditors_per_category['cov_cat'] == '≥ 80%') | 
        (avg_creditors_per_category['cov_cat'] == '100%'), 
        'num_creditors'
        ].values[0]
    average_creditors_50_to_80 = avg_creditors_per_category.loc[
        (avg_creditors_per_category['cov_cat'] == '≥ 80%') |
        (avg_creditors_per_category['cov_cat'] == '100%'),
        'num_creditors'
        ].values[0]
    average_creditors_ge_80 = avg_creditors_per_category.loc[avg_creditors_per_category['cov_cat'] == '100%', 'num_creditors'].values[0]
    average_creditors_per_lot = data.groupby('cluster')['Кредитор'].nunique().mean()

    num_lots = data['cluster'].nunique()
    average_price_per_lot = data.groupby('cluster')['Сумма во ВВ'].sum().mean()

    MQ = 0.5 * ((1 - num_lots / manual_lot_count) + (1 - manual_avg_price / average_price_per_lot))

    MS = (2 * average_creditors_le_50 + 3 * average_creditors_50_to_80 + 4 * average_creditors_ge_80) / average_creditors_per_lot

    print(f"MQ metric is: {MQ}")
    print(f"MS metric is: {MS}")


  2%|▏         | 1/49 [00:01<01:10,  1.46s/it]

MQ metric is: 0.9896415400021213
MS metric is: 0.2734834960306909


  4%|▍         | 2/49 [00:04<01:57,  2.50s/it]

MQ metric is: 0.9792830800042427
MS metric is: 0.4886583011583012


  6%|▌         | 3/49 [00:08<02:27,  3.21s/it]

...

MQ metric is: 0.5131523800997038
MS metric is: 6.090010306476722


 98%|█████████▊| 48/49 [20:32<00:44, 44.74s/it]

MQ metric is: 0.502793920101825
MS metric is: 6.164880938133228


100%|██████████| 49/49 [21:16<00:00, 26.05s/it]

MQ metric is: 0.4924354601039464
MS metric is: 6.2130951057035375


In [22]:
k = 3300

kmeans = KMeans(n_clusters=k, random_state=42)
data['cluster'] = kmeans.fit_predict(data_scaled)

# data['Материал_Класс'] = data['Материал'].astype(str) + '-' + data['Класс'].astype(str)
data['Материал_Класс'] = data['Класс'].astype(str)

# Step 2: Add a column for the number of unique "Материал_Класс" pairs in each cluster
data['pairs_per_clust'] = data.groupby('cluster')['Материал_Класс'].transform('nunique')

# Step 3: Add a column to count the number of unique "Материал_Класс" pairs covered by each creditor in each cluster
data['pairs_by_cred'] = data.groupby(['Кредитор', 'cluster'])['Материал_Класс'].transform('nunique')

# Step 4: Calculate the percentage of pairs each creditor covers in each cluster
data['perc_cov'] = (data['pairs_by_cred'] / data['pairs_per_clust']) * 100

def categorize_coverage(percentage):
    if 80 > percentage >= 50:
        return '≥ 50%'
    elif 100 > percentage >= 80:
        return '≥ 80%'
    elif percentage == 100:
        return '100%'

data['cov_cat'] = data['perc_cov'].apply(categorize_coverage)

# Group by cluster and calculate the number of creditors in each category
creditor_counts_by_category = data.groupby(['cluster', 'cov_cat'])['Кредитор'].nunique().reset_index()
cred_count = data.groupby(['cluster', 'cov_cat'])['Кредитор'].transform('nunique')
data['num_creditors'] = cred_count
creditor_counts_by_category.columns = ['cluster', 'cov_cat', 'num_creditors']
avg_creditors_per_category = creditor_counts_by_category.groupby('cov_cat')['num_creditors'].mean().reset_index()

average_creditors_le_50 = avg_creditors_per_category.loc[
    (avg_creditors_per_category['cov_cat'] == '≥ 50%') | 
    (avg_creditors_per_category['cov_cat'] == '≥ 80%') | 
    (avg_creditors_per_category['cov_cat'] == '100%'), 
    'num_creditors'
    ].values[0]
average_creditors_50_to_80 = avg_creditors_per_category.loc[
    (avg_creditors_per_category['cov_cat'] == '≥ 80%') |
    (avg_creditors_per_category['cov_cat'] == '100%'),
    'num_creditors'
    ].values[0]
average_creditors_ge_80 = avg_creditors_per_category.loc[avg_creditors_per_category['cov_cat'] == '100%', 'num_creditors'].values[0]
average_creditors_per_lot = data.groupby('cluster')['Кредитор'].nunique().mean()

num_lots = data['cluster'].nunique()
average_price_per_lot = data.groupby('cluster')['Сумма во ВВ'].sum().mean()

MQ = 0.5 * ((1 - num_lots / manual_lot_count) + (1 - manual_avg_price / average_price_per_lot))

MS = (2 * average_creditors_le_50 + 3 * average_creditors_50_to_80 + 4 * average_creditors_ge_80) / average_creditors_per_lot

print(f"MQ metric is: {MQ}")
print(f"MS metric is: {MS}")

MQ metric is: 0.6581708200700047
MS metric is: 5.022751644681082
